# Dependency Inversion Principle

La dependencia debe estar en abstracciones, no en concreciones. Los módulos de alto nivel no deben depender de módulos de bajo nivel. Ambos deberían depender de abstracciones. Las abstracciones no deben depender de los detalles. Los detalles deben depender de abstracciones.

Llega un punto en el desarrollo de software en el que nuestra aplicación estará compuesta en gran parte por módulos. Cuando esto sucede, tenemos que aclarar las cosas usando la inyección de dependencia. Componentes de alto nivel dependiendo de los componentes de bajo nivel para funcionar.

In [7]:
class XMLHttpService():
    pass

class Http:
    def __init__(self, xml_http_service: XMLHttpService):
        self.xml_http_service = xml_http_service
    
    def get(self, url: str, options: dict):
        self.xml_http_service.request(url, 'GET')

    def post(self, url, options: dict):
        self.xml_http_service.request(url, 'POST')

        

Aquí, `Http` es el componente de alto nivel mientras que `XMLHttpService` es el componente de bajo nivel. Este diseño viola **DIP**: los módulos de alto nivel no deben depender de módulos de bajo nivel. Debería depender de su abstracción.

Esta clase `Http` está obligada a depender de la clase `XMLHttpService`. Si tuviéramos que cambiar el servicio de conexión Http, tal vez queramos conectarnos a Internet a través de *cURL* o incluso *simular* el servicio http. Tendremos que movernos minuciosamente por todas las instancias de Http para editar el código y esto viola el principio **OCP**.

La clase `Http` debería preocuparse menos por el tipo de servicio Http que está utilizando. Hacemos una interfaz de conexión:

In [8]:
class Connection:
    def request(self, url: str, options: dict):
        raise NotImplementedError

        

La interfaz `Connection` tiene un método de solicitud. Con esto, pasamos un argumento de tipo `Connection` a nuestra clase `Http`:

In [9]:
class Http:
    def __init__(self, http_connection: Connection):
        self.http_connection = http_connection
    
    def get(self, url: str, options: dict):
        self.http_connection.request(url, 'GET')

    def post(self, url, options: dict):
        self.http_connection.request(url, 'POST')

        

Así que ahora, sin importar el tipo de servicio de conexión Http pasado a `Http`, puede conectarse fácilmente a una red sin molestarse en saber el tipo de conexión de red.

Ahora podemos volver a implementar nuestra clase `XMLHttpService` para implementar la interfaz de conexión:

In [11]:
class XMLHttpService(Connection):
    def request(self, url: str, options:dict):
        pass  

Podemos crear muchos tipos de conexión Http y pasarlos a nuestra clase `Http` sin ningún problema por los errores.

In [12]:
class NodeHttpService(Connection):
    def request(self, url: str, options:dict):
        pass

class MockHttpService(Connection):
    def request(self, url: str, options:dict):
        pass

Ahora, podemos ver que tanto los módulos de alto nivel como los de bajo nivel dependen de abstracciones. La clase `Http` (módulo de alto nivel) depende de la interfaz de `Connection` (abstracción) y los tipos de servicio Http (módulos de bajo nivel), a su vez, depende de la interfaz de `Connection` (abstracción).

Además, este **DIP** nos obligará a no violar el principio de sustitución de **Liskov**: los tipos de conexión `Node-XML-MockHttpService` son sustituibles por su tipo de conexión principal.